In [45]:
import pandas as pd
import re

# Function to normalize team names for fuzzy matching
def normalize_team_name(name):
    if pd.isna(name):
        return ""
    return re.sub(r'[^a-z0-9]', '', name.lower())

# Load the datasets
rankings_df = pd.read_csv('ncaa_rankings_W_2025.csv')
stats_df = pd.read_csv('select_sun_W_stats2024.csv')

# Keep only the columns we need from rankings
rankings_df = rankings_df[['Rank', 'School']]

Successfully matched 358 out of 360 teams (99.4%)

Unmatched teams (2):
['IUPUI', 'St_Francis_PA']


/var/folders/pb/p1rzn9f54njd0b95nz_lqwy40000gn/T/ipykernel_19801/3188335096.py:253: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmatched['NormalizedMappedName'] = unmatched['MappedTeamName'].apply(normalize_team_name)
/var/folders/pb/p1rzn9f54njd0b95nz_lqwy40000gn/T/ipykernel_19801/3188335096.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmatched['MatchedInfo'] = unmatched['NormalizedMappedName'].map(find_match)
/var/folders/pb/p1rzn9f54njd0b95nz_lqwy40000gn/T/ipykernel_19801/3188335096.py:267: 

### Map the teamnames from stats of the files to the ranking file

RECOMMENDATION: Utilize LLM tools to match the files initially and do the rest by hand (if LLMs don't work for the stragglers)

In [ ]:
# Create a mapping dictionary for team names that don't match directly
name_mapping = {
    "S_Dakota_St": "South Dakota St.",
    "Murray_St": "Murray St.",
    "USC": "Southern California",
    "Boise_St": "Boise St.",
    "UAB": "UAB",
    "Texas_Tech": "Texas Tech",
    "Montana_St": "Montana St.",
    "Colorado_St": "Colorado St.",
    "Loyola-Chicago": "Loyola Chicago",
    "St_Mary's_CA": "Saint Mary's (CA)",
    "North_Carolina": "North Carolina",
    "Wake_Forest": "Wake Forest",
    "San_Francisco": "San Francisco",
    "Connecticut": "UConn",
    "San_Diego_St": "San Diego St.",
    "Miami_FL": "Miami (FL)",
    "New_Mexico_St": "New Mexico St.",
    "Michigan_St": "Michigan St.",
    "St_Louis": "Saint Louis",
    "Texas_A&M": "Texas A&M;",
    "North_Texas": "North Texas",
    "UNC_Wilmington": "UNCW",
    "Monmouth_NJ": "Monmouth",
    "Kent": "Kent St.",
    "Grand_Canyon": "Grand Canyon",
    "Norfolk_St": "Norfolk St.",
    "MTSU": "Middle Tenn.",
    "N_Dakota_St": "North Dakota St.",
    "Boston_Univ": "Boston U.",
    "Morehead_St": "Morehead St.",
    "Wright_St": "Wright St.",
    "St_Bonaventure": "St. Bonaventure",
    "Iowa_St": "Iowa St.",
    "TAM_C._Christi": "A&M-Corpus; Christi",
    "Jacksonville_St": "Jacksonville St.",
    "PFW": "Purdue Fort Wayne",
    "Abilene_Chr": "Abilene Christian",
    "CS_Fullerton": "Cal St. Fullerton",
    "Weber_St": "Weber St.",
    "Ohio_St": "Ohio St.",
    "FGCU": "FGCU",
    "Washington_St": "Washington St.",
    "Utah_Valley": "Utah Valley",
    "Long_Beach_St": "Long Beach St.",
    "Northern_Iowa": "UNI",
    "Fresno_St": "Fresno St.",
    "Mississippi_St": "Mississippi St.",
    "St_Peter's": "Saint Peter's",
    "Southern_Utah": "Southern Utah",
    "N_Kentucky": "Northern Ky.",
    "Cleveland_St": "Cleveland St.",
    "N_Colorado": "Northern Colo.",
    "Texas_St": "Texas St.",
    "TX_Southern": "Texas Southern",
    "SF_Austin": "SFA",
    "Col_Charleston": "Col. of Charleston",
    "Utah_St": "Utah St.",
    "UMBC": "UMBC",
    "Alcorn_St": "Alcorn",
    "Nicholls_St": "Nicholls",
    "WKU": "Western Ky.",
    "St_John's": "St. John's (NY)",
    "Florida_St": "Florida St.",
    "UNC_Greensboro": "UNC Greensboro",
    "Youngstown_St": "Youngstown St.",
    "FL_Atlantic": "Fla. Atlantic",
    "Appalachian_St": "App State",
    "E_Washington": "Eastern Wash.",
    "Oral_Roberts": "Oral Roberts",
    "West_Virginia": "West Virginia",
    "Sam_Houston_St": "Sam Houston",
    "East_Carolina": "East Carolina",
    "UC_Santa_Barbara": "UC Santa Barbara",
    "Gardner_Webb": "Gardner-Webb",
    "S_Illinois": "Southern Ill.",
    "Southern_Univ": "Southern U.",
    "Georgia_St": "Georgia St.",
    "Massachusetts": "Massachusetts",
    "Missouri_KC": "Kansas City",
    "New_Orleans": "New Orleans",
    "Cal_Baptist": "California Baptist",
    "Arkansas_St": "Arkansas St.",
    "South_Alabama": "South Alabama",
    "Wichita_St": "Wichita St.",
    "LIU_Brooklyn": "LIU",
    "SE_Louisiana": "Southeastern La.",
    "Oklahoma_St": "Oklahoma St.",
    "Arizona_St": "Arizona St.",
    "UC_Riverside": "UC Riverside",
    "Kansas_St": "Kansas St.",
    "Rhode_Island": "Rhode Island",
    "W_Illinois": "Western Ill.",
    "George_Mason": "George Mason",
    "Penn_St": "Penn St.",
    "SUNY_Albany": "UAlbany",
    "UNC_Asheville": "UNC Asheville",
    "Mt_St_Mary's": "Mount St. Mary's",
    "New_Hampshire": "New Hampshire",
    "Detroit": "Detroit Mercy",
    "Florida_Intl": "FIU",
    "Boston_College": "Boston College",
    "Ball_St": "Ball St.",
    "UC_Irvine": "UC Irvine",
    "IL_Chicago": "UIC",
    "James_Madison": "James Madison",
    "Mississippi": "Ole Miss",
    "San_Diego": "San Diego",
    "Loyola_MD": "Loyola Maryland",
    "Alabama_A&M": "Alabama A&M;",
    "G_Washington": "George Washington",
    "Valparaiso": "Valparaiso",
    "S_Carolina_St": "South Carolina St.",
    "UC_Davis": "UC Davis",
    "MA_Lowell": "UMass Lowell",
    "Tennessee_St": "Tennessee St.",
    "SE_Missouri_St": "Southeast Mo. St.",
    "Portland_St": "Portland St.",
    "SC_Upstate": "USC Upstate",
    "Coastal_Car": "Coastal Carolina",
    "NC_Central": "N.C. Central",
    "Old_Dominion": "Old Dominion",
    "Jackson_St": "Jackson St.",
    "St_Joseph's_PA": "Saint Joseph's",
    "New_Mexico": "New Mexico",
    "Illinois_St": "Illinois St.",
    "Bowling_Green": "Bowling Green",
    "High_Point": "High Point",
    "Miami_OH": "Miami (OH)",
    "NC_State": "NC State",
    "UC_San_Diego": "UC San Diego",
    "Tarleton_St": "Tarleton St.",
    "La_Salle": "La Salle",
    "Air_Force": "Air Force",
    "Georgia_Tech": "Georgia Tech",
    "Florida_A&M": "Florida A&M;",
    "Ga_Southern": "Ga. Southern",
    "ULM": "ULM",
    "W_Carolina": "Western Caro.",
    "Loy_Marymount": "LMU (CA)",
    "American_Univ": "American",
    "UT_Arlington": "UT Arlington",
    "WI_Milwaukee": "Milwaukee",
    "Morgan_St": "Morgan St.",
    "Kennesaw": "Kennesaw St.",
    "N_Illinois": "NIU",
    "NC_A&T": "N.C. A&T;",
    "Tennessee_Tech": "Tennessee Tech",
    "St_Francis_NY": "St. Francis Brooklyn",
    "CS_Sacramento": "Sacramento St.",
    "SIUE": "SIUE",
    "E_Kentucky": "Eastern Ky.",
    "E_Michigan": "Eastern Mich.",
    "Coppin_St": "Coppin St.",
    "Cent_Arkansas": "Central Ark.",
    "Alabama_St": "Alabama St.",
    "Utah_Tech": "Utah Tech",
    "Sacred_Heart": "Sacred Heart",
    "Prairie_View": "Prairie View",
    "MD_E_Shore":"UMES",
    "Central_Conn": "Central Conn. St.",
    "Indiana_St": "Indiana St.",
    "North_Florida": "North Florida",
    "St_Francis_PA": "Saint Francis (PA)",
    "Bethune-Cookman": "Bethune-Cookman",
    "South_Florida": "South Fla.",
    "Holy_Cross": "Holy Cross",
    "Robert_Morris": "Robert Morris",
    "Houston_Chr": "Houston Christian",
    "Idaho": "Idaho",
    "Chicago_St": "Chicago St.",
    "St_Thomas_MN": "St. Thomas (MN)",
    "C_Michigan": "Central Mich.",
    "Stetson": "Stetson",
    "San_Jose_St": "San Jose St.",
    "UT_San_Antonio": "UTSA",
    "Northern_Arizona": "Northern Ariz.",
    "McNeese_St": "McNeese",
    "Duquesne": "Duquesne",
    "UTRGV": "UTRGV",
    "Ark_Little_Rock": "Little Rock",
    "Cal_Poly": "Cal Poly",
    "TN_Martin": "UT Martin",
    "Northwestern_LA": "Northwestern St.",
    "CS_Northridge": "CSUN",
    "CS_Bakersfield": "CSU Bakersfield",
    "W_Michigan": "Western Mich.",
    "Idaho_St": "Idaho St.",
    "Ark_Pine_Bluff": "Ark.-Pine Bluff",
    "Incarnate_Word": "UIW",
    "Southern_Miss": "Southern Miss.",
    "North_Alabama": "North Ala.",
    "Evansville": "Evansville",
    "William_&_Mary": "William & Mary",
    "F_Dickinson": "FDU",
    "NE_Omaha": "Omaha",
    "North_Dakota": "North Dakota",
    "WI_Green_Bay": "Green Bay",
    "Oregon_St": "Oregon St.",
    "E_Illinois": "Eastern Ill.",
    "Maine": "Maine",
    "Columbia": "Columbia",
    "Charleston_So": "Charleston So.",
    "MS_Valley_St": "Mississippi Val.",
    "IUPUI": "IUPUI",
    "Lamar": "Lamar University",
    "Delaware_St": "Delaware St.",
    "Seattle": "Seattle U",
    "Army" : "Army West Point",
    "Citadel": "The Citadel",
    "Southern_Indiana": "Southern Ind.",
    "St_Francis_PA": "Saint Francis (PA)", 
    "IUPUI": "IUPUI",
    "St_Francis_NY" : "St. Francis Brooklyn",
    'Hartford':'Hartford'
}



### Merge the tables

In [ ]:
# Apply the mapping to create a new column with standardized names
stats_df['MappedTeamName'] = stats_df['TeamName'].map(lambda x: name_mapping.get(x, x))

# First attempt to merge based on exact matches
merged_df = pd.merge(
    stats_df,
    rankings_df,
    left_on='MappedTeamName',
    right_on='School',
    how='left'
)

# For remaining unmatched teams, try normalized matching
unmatched = merged_df[merged_df['Rank'].isna()]
matched = merged_df[~merged_df['Rank'].isna()]

# Create normalized columns for matching
rankings_df['NormalizedName'] = rankings_df['School'].apply(normalize_team_name)
unmatched['NormalizedMappedName'] = unmatched['MappedTeamName'].apply(normalize_team_name)

# Create a dictionary mapping normalized names to original School and Rank
norm_to_orig = dict(zip(rankings_df['NormalizedName'], 
                        zip(rankings_df['School'], rankings_df['Rank'])))

# Function to find match based on normalized name
def find_match(norm_name):
    if norm_name in norm_to_orig:
        return norm_to_orig[norm_name]
    return None, None

# Apply matching
unmatched['MatchedInfo'] = unmatched['NormalizedMappedName'].map(find_match)
unmatched['MatchedSchool'] = unmatched['MatchedInfo'].map(lambda x: x[0] if x and x[0] is not None else None)
unmatched['MatchedRank'] = unmatched['MatchedInfo'].map(lambda x: x[1] if x and x[1] is not None else None)

# Combine the results
unmatched_final = unmatched.drop(['School', 'Rank', 'NormalizedMappedName', 'MatchedInfo'], axis=1, errors='ignore')
unmatched_final = unmatched_final.rename(columns={'MatchedSchool': 'School', 'MatchedRank': 'Rank'})

matched_final = matched.drop(['NormalizedMappedName', 'MatchedInfo', 'MatchedSchool', 'MatchedRank'], axis=1, errors='ignore')

# Concatenate the matched and newly matched data
final_df = pd.concat([matched_final, unmatched_final], ignore_index=True)

# Add the Rank as 'Ranking' column but keep only one team name
# Use the 'School' from rankings when available, otherwise keep original TeamName
final_df['TeamNameFinal'] = final_df['School'].fillna(final_df['TeamName'])

# Select all columns from stats_df plus Rank, replacing TeamName with our final version
cols_to_keep = [col for col in stats_df.columns if col != 'TeamName'] + ['TeamNameFinal', 'Rank']
result_df = final_df[cols_to_keep]

# Rename the columns for clarity
result_df = result_df.rename(columns={
    'TeamNameFinal': 'TeamName',  # Replace the original TeamName with our merged version
    'Rank': 'Ranking'             # Rename Rank to Ranking
})

# Drop the temporary MappedTeamName column that was used for matching
result_df = result_df.drop('MappedTeamName', axis=1)

# Convert Ranking to numeric
result_df['Ranking'] = pd.to_numeric(result_df['Ranking'], errors='coerce')

# Sort by Ranking
result_df = result_df.sort_values('Ranking')



# Check how many teams were successfully matched
matched_count = result_df['Ranking'].notna().sum()
total_count = len(result_df)
print(f"Successfully matched {matched_count} out of {total_count} teams ({matched_count/total_count:.1%})")

# Show a few sample teams that couldn't be matched
unmatched_teams = result_df[result_df['Ranking'].isna()]['TeamName'].tolist()
print(f"\nUnmatched teams ({len(unmatched_teams)}):")
if unmatched_teams:
    print(unmatched_teams[:10])  # Show just the first 10 unmatched teams

In [46]:
#result_df
has_na = result_df.isna().any().any()
print(f"Has NA values: {has_na}")
# Output: Has NA values: True

# Check for NA values in each column
has_na_col = result_df.isna().any()
print(f"Has NA values per column:\n{has_na_col}")
#result_df = result_df.dropna()
#has_na = result_df.isna().any().any()
#print(f"Has NA values: {has_na}")

Has NA values: True
Has NA values per column:
TeamID                   False
HomeWins                 False
HomeLosses               False
AwayWins                 False
AwayLosses               False
NeutralWins              False
NeutralLosses            False
TotalWins                False
TotalLosses              False
OpponentHomeWins         False
OpponentHomeLosses       False
OpponentAwayWins         False
OpponentAwayLosses       False
OpponentNeutralWins      False
OpponentNeutralLosses    False
OpponentTotalWins        False
OpponentTotalLosses      False
TeamName                 False
Ranking                   True
dtype: bool


In [47]:
# Save the merged dataset
#result_df.to_csv('merged_ncaa_data.csv', index=False)
result_df

,TeamID,HomeWins,HomeLosses,AwayWins,AwayLosses,NeutralWins,NeutralLosses,TotalWins,TotalLosses,OpponentHomeWins,OpponentHomeLosses,OpponentAwayWins,OpponentAwayLosses,OpponentNeutralWins,OpponentNeutralLosses,OpponentTotalWins,OpponentTotalLosses,TeamName,Ranking
9,57,13,1,10,3,5,1,28,5,240,77,134,118,56,31,430,226,UConn,1.0
345,270,15,0,12,0,5,0,32,0,272,105,139,155,64,43,475,303,South Carolina,2.0
1,298,15,2,9,2,6,0,30,4,261,105,128,167,58,48,447,320,Texas,3.0
24,316,14,1,7,4,4,1,25,6,228,92,127,125,48,29,403,246,UCLA,4.0
347,217,11,3,11,2,4,1,26,6,250,117,125,158,46,48,421,323,Notre Dame,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,146,5,9,3,13,0,0,8,22,119,129,75,213,4,23,198,365,LIU,360.0
342,249,2,8,0,18,0,2,2,28,145,97,111,194,27,34,283,325,South Carolina St.,361.0
308,69,4,7,2,12,0,1,6,20,130,95,99,142,33,27,262,264,Delaware St.,362.0
357,128,5,9,2,13,0,1,7,23,154,94,103,155,25,27,282,276,IUPUI,NaN


### Add the "Quadrant" for each team

In [48]:
import numpy as np
conditions = [
    (result_df['Ranking'] <= 50),
    (result_df['Ranking'] > 50) & (result_df['Ranking'] <= 100),
    (result_df['Ranking'] > 100) & (result_df['Ranking'] <= 200),
    (result_df['Ranking'] > 200),
    (result_df["Ranking"].isna())
]
values = [1, 2, 3, 4, 4] # Look into the data to set the NaN values, may not always be 4
result_df['Quadrant'] = np.select(conditions, values)

# Example usage to add the quadrant column to a dataframe
result_df

,TeamID,HomeWins,HomeLosses,AwayWins,AwayLosses,NeutralWins,NeutralLosses,TotalWins,TotalLosses,OpponentHomeWins,OpponentHomeLosses,OpponentAwayWins,OpponentAwayLosses,OpponentNeutralWins,OpponentNeutralLosses,OpponentTotalWins,OpponentTotalLosses,TeamName,Ranking,Quadrant
9,57,13,1,10,3,5,1,28,5,240,77,134,118,56,31,430,226,UConn,1.0,1
345,270,15,0,12,0,5,0,32,0,272,105,139,155,64,43,475,303,South Carolina,2.0,1
1,298,15,2,9,2,6,0,30,4,261,105,128,167,58,48,447,320,Texas,3.0,1
24,316,14,1,7,4,4,1,25,6,228,92,127,125,48,29,403,246,UCLA,4.0,1
347,217,11,3,11,2,4,1,26,6,250,117,125,158,46,48,421,323,Notre Dame,5.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,146,5,9,3,13,0,0,8,22,119,129,75,213,4,23,198,365,LIU,360.0,4
342,249,2,8,0,18,0,2,2,28,145,97,111,194,27,34,283,325,South Carolina St.,361.0,4
308,69,4,7,2,12,0,1,6,20,130,95,99,142,33,27,262,264,Delaware St.,362.0,4
357,128,5,9,2,13,0,1,7,23,154,94,103,155,25,27,282,276,IUPUI,NaN,4


In [49]:
result_df.to_csv('ncaa_data_W_24_to_25.csv', index=False)

In [53]:
df1 = pd.read_csv('ncaa_data_W_23_to_24.csv')
df2 = pd.read_csv('ncaa_data_W_24_to_25.csv')


In [54]:
ncaa_data_all = pd.concat([df1, df2], axis=0)
ncaa_data_all.to_csv('ncaa_data_W_all.csv', index=False)